## インポート

In [1]:
from config import (NUMBERS,
                                  NUMBERS3,
                                  NUMBERS4,
                                  NUMBERS3_RESULT,
                                  NUMBERS4_RESULT,
                                  NUMBERS3_VECTORS)
import utils

import sys
import re
import difflib
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

plt.rcParams['font.family'] = 'AppleGothic'


%load_ext autoreload
%autoreload 2

##### 調査 その1

In [3]:
X = []
Y = []
Z = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0
key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

"""
先勝・友引・先負・仏滅・大安・赤口
"""
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        # if len(list(set(_y) & set(_v))) >= 2:
        if len(list(set(_y) & set(_v))) >= 2 and _z == "先負":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_time, _y, _v)
            # print(_format)
            pass
_sum     = sum([key_0, key_1, key_2, key_3, key_4, key_5])
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)

data = {
    "base": [key_0, key_1, key_2, key_3, key_4, key_5],
    "old_week": [key_6, key_7, key_8, key_9, key_10, key_11]
}
_data = pd.DataFrame(data)
print(_format1)
print(_format2)

# sns.countplot(x="old_week", data=_data)


前回:18個, 前々回:28個, 前回曜日:27個, 前々回曜日:18個, 前回六曜:23個, 前々回六曜:20個
赤口:0個, 仏滅:0個, 友引:0個 先勝:0個, 先負:134個, 大安:0個


### 調査2

In [19]:
X = []
Y = []
Z = []
LOCK = {}
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time]
    lock = []
    hook = []
    key_0 = 0
    key_1 = 0
    key_2 = 0
    key_3 = 0
    key_4 = 0
    key_5 = 0
    key_6 = 0
    key_7 = 0
    key_8 = 0
    key_9 = 0
    for _n in _x:
        if _time == _time:
            _h = list(_n)
            for _j in _h:
                if _j == '0':
                    key_0 += 1
                elif _j == '1':
                    key_1 += 1
                elif _j == '2':
                    key_2 += 1
                elif _j == '3':
                    key_3 += 1
                elif _j == '4':
                    key_4 += 1
                elif _j == '5':
                    key_5 += 1
                elif _j == '6':
                    key_6 += 1
                elif _j == '7':
                    key_7 += 1
                elif _j == '8':
                    key_8 += 1
                elif _j == '9':
                    key_9 += 1
            lock.append(list(_n))
    data = {
        "0": str(int(key_0) + 1),
        "1": str(int(key_1) + 1),
        "2": str(int(key_2) + 1),
        "3": str(int(key_3) + 1),
        "4": str(int(key_4) + 1),
        "5": str(int(key_5) + 1),
        "6": str(int(key_6) + 1),
        "7": str(int(key_7) + 1),
        "8": str(int(key_8) + 1),
        "9": str(int(key_9) + 1),
    }
    LOCK.update({ _time: [key_0, key_1, key_2, key_3, key_4, key_5, key_6, key_7, key_8, key_9]})
    Z.append(data)
    X.append(lock)
    
lock = pd.DataFrame(LOCK)

#### 抽選数

In [ ]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS3_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x0 = NUMBERS3_RESULT[_i][_time][0]
    _x1 = NUMBERS3_RESULT[_i][_time][1]
    _x2 = NUMBERS3_RESULT[_i][_time][2]
    _x3 = NUMBERS3_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

### 設定

In [110]:
OLD_WEEK = "先勝"
TYPE            = "-"

#### 特徴量

In [111]:
VEC = utils.get_vectors(OLD_WEEK, TYPE, NUMBERS3_VECTORS)

#### セット

In [112]:
X       = []
Y       = []
TIME = []
k1 = VEC[0]
k2 = VEC[1]
k3 = VEC[2]

#### 計算

In [203]:
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][k1]
    _x2 = NUMBERS3[_i][_time][k2]
    _x3 = NUMBERS3[_i][_time][k3]
    _z = NUMBERS[_i][4]
    if not len(list(set(_y) & set(_x1))) >= 1 and not len(list(set(_y) & set(_x2))) >= 1 and _z == OLD_WEEK:
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)
        pass

### 予測スタート

###### 20180709

In [204]:
pkl_file = "data/numbers3_20180710.pkl"

###### + or -

In [205]:
data = [float("061"), float("256"), float("751")]

In [206]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [207]:
len(y)

910

In [208]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [209]:
px = x
py = model.predict(px)

In [210]:
NUM = []
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[2]) & set(_py))) >= 2:
                _format = "第{0}回: {1}<-{2}   予測:{3}".format(TIME[_i], _v[2], py[_i], LOCK[_v[0]])
                print(_format)
                NUM.append([TIME[_i], _v[2], py[_i], LOCK[_v[0]]])
                pass
print(len(NUM))

NameError: name 'LOCK' is not defined

## 決定係数

In [211]:
r2_score(y, py)

0.00903336105866015

In [212]:
model.score(x, y)

0.00903336105866015

In [213]:
model.coef_[0]

0.0035182071666690897

In [214]:
joblib.dump(model, pkl_file)

['data/numbers3_20180710.pkl']

In [215]:
clf = joblib.load(pkl_file)

In [216]:
processor = clf.predict([data])

In [217]:
processor

array([515.0514047])